In [ ]:
tools: Optional[List[Union[Callable, Tuple[Callable, Type[BaseModel], Optional[Type[BaseModel]]], Dict[str, Any]]]] = None,


class ToolDefinition:
    """Helper class for defining tools with Pydantic models."""
    
    @staticmethod
    def from_function(
        function: Callable,
        params_model: Optional[Type[BaseModel]] = None,
        response_model: Optional[Type[BaseModel]] = None,
        name: Optional[str] = None,
        description: Optional[str] = None
    ) -> Tuple[Callable, Dict[str, Any]]:
        """
        Create a tool definition from a function and optional Pydantic models.
        
        Args:
            function: The function implementation
            params_model: Optional Pydantic model for parameters
            response_model: Optional Pydantic model for response
            name: Optional custom name
            description: Optional custom description
            
        Returns:
            Tuple of (function, schema)
        """
        if params_model:
            # Use Pydantic model for schema
            import inspect
            
            # Get function metadata
            func_name = name or function.__name__
            func_description = description or inspect.getdoc(function) or ""
            
            # Convert Pydantic models to Gemini schemas
            params_schema = GeminiUtils.pydantic_to_schema(params_model)
            
            # Create function definition
            schema = {
                "name": func_name,
                "description": func_description,
                "parameters": params_schema
            }
            
            # Add response schema if provided
            if response_model:
                response_schema = GeminiUtils.pydantic_to_schema(response_model)
                schema["response"] = response_schema
                
            return (function, schema)
        else:
            # Let GeminiUtils extract schema from function signature
            schema = GeminiUtils.python_to_function(function, name, description)
            return (function, schema)
        
        # Process tools to standardized format
        processed_tools = []
        tool_callables = {}
        
        if tools:
            for tool in tools:
                if isinstance(tool, tuple) and len(tool) >= 2 and callable(tool[0]):
                    # Case: (function, params_model, [response_model]) tuple
                    function = tool[0]
                    params_model = tool[1]
                    response_model = tool[2] if len(tool) > 2 else None
                    
                    func, schema = ToolDefinition.from_function(
                        function=function,
                        params_model=params_model,
                        response_model=response_model
                    )
                    
                    processed_tools.append(schema)
                    tool_callables[schema["name"]] = func
                    
                elif callable(tool):
                    # Case: Plain function
                    schema = GeminiUtils.python_to_function(tool)
                    processed_tools.append(schema)
                    tool_callables[schema["name"]] = tool
                    
                elif isinstance(tool, dict) and "name" in tool:
                    # Case: Pre-made schema
                    processed_tools.append(tool)
                    # Note: Function implementation must be provided separately
                    
                else:
                    logger.warning(f"Ignoring unsupported tool type: {type(tool)}")

In [ ]:
"""
Simplified utility functions for working with Gemini API formats.
"""

import inspect
import json
from typing import Dict, List, Any, Optional, Callable, Union, get_type_hints, get_origin, get_args
from loguru import logger


class GeminiUtils:
    """
    Utilities for working with Gemini API formats and conversions.
    """
    
    @staticmethod
    def pydantic_to_schema(schema: Any) -> Dict[str, Any]:
        """
        Convert Pydantic schema to Gemini API schema format.
        
        Args:
            schema: Pydantic schema to convert
            
        Returns:
            Schema in Gemini API format
        """
        # If it's a Pydantic model, get its JSON schema
        if hasattr(schema, "model_json_schema"):
            json_schema = schema.model_json_schema()
            # Convert JSON schema to Gemini format
            return GeminiUtils._convert_json_schema_to_gemini(json_schema)
        
        # If not a Pydantic model, return as is (assuming it's already in correct format)
        return schema
        
    @staticmethod
    def _convert_json_schema_to_gemini(pydantic_schema: Dict[str, Any], root_schema: Dict[str, Any] = None) -> Dict[str, Any]:
        """
        Convert JSON schema (from Pydantic) to Gemini schema format.
        
        Args:
            pydantic_schema: JSON schema to convert
            root_schema: Root schema for resolving references
            
        Returns:
            Gemini API compatible schema
        """
        # Store the root schema for resolving references if not passed in
        if root_schema is None:
            root_schema = pydantic_schema
        
        gemini_schema = {}
        
        # Handle $ref references
        if "$ref" in pydantic_schema:
            ref_path = pydantic_schema["$ref"]
            if ref_path.startswith("#/"):
                # Split the path into components
                parts = ref_path.lstrip("#/").split("/")
                
                # Navigate through the schema to find the referenced object
                current = root_schema
                for part in parts:
                    if part in current:
                        current = current[part]
                    else:
                        break
                
                if current != root_schema:
                    return GeminiUtils._convert_json_schema_to_gemini(current, root_schema)
        
        # Handle Optional types (which Pydantic often represents with anyOf)
        if "anyOf" in pydantic_schema:
            null_present = any(schema.get("type") == "null" for schema in pydantic_schema["anyOf"])
            
            if null_present:
                for schema in pydantic_schema["anyOf"]:
                    if schema.get("type") != "null":
                        description = pydantic_schema.get("description")
                        title = pydantic_schema.get("title")
                        
                        non_null_schema = GeminiUtils._convert_json_schema_to_gemini(schema, root_schema)
                        gemini_schema.update(non_null_schema)
                        gemini_schema["nullable"] = True
                        
                        if description:
                            gemini_schema["description"] = description
                        elif title:
                            gemini_schema["description"] = title
                        
                        return gemini_schema
        
        # Map basic type
        if "type" in pydantic_schema:
            type_mapping = {
                "string": "STRING",
                "number": "NUMBER",
                "integer": "INTEGER",
                "boolean": "BOOLEAN",
                "array": "ARRAY",
                "object": "OBJECT"
            }
            gemini_schema["type"] = type_mapping.get(pydantic_schema["type"], pydantic_schema["type"])
        
        # Map title and description
        if "description" in pydantic_schema:
            gemini_schema["description"] = pydantic_schema["description"]
        elif "title" in pydantic_schema:
            gemini_schema["description"] = pydantic_schema["title"]
        
        # Map enum values
        if "enum" in pydantic_schema:
            gemini_schema["enum"] = pydantic_schema["enum"]
            if "type" not in gemini_schema and pydantic_schema["enum"]:
                gemini_schema["type"] = "STRING"
        
        # Map properties for objects
        if "properties" in pydantic_schema:
            gemini_schema["properties"] = {}
            for prop_name, prop_schema in pydantic_schema["properties"].items():
                gemini_schema["properties"][prop_name] = GeminiUtils._convert_json_schema_to_gemini(prop_schema, root_schema)
        
        # Map required fields
        if "required" in pydantic_schema:
            gemini_schema["required"] = pydantic_schema["required"]
        
        # Map items for arrays
        if "items" in pydantic_schema:
            gemini_schema["items"] = GeminiUtils._convert_json_schema_to_gemini(pydantic_schema["items"], root_schema)
        
        return gemini_schema
    
    @staticmethod
    def python_to_function(function: Callable) -> Dict[str, Any]:
        """
        Convert Python function to Gemini API function declaration format.
        
        Args:
            function: Function to convert
            
        Returns:
            Function declaration in Gemini API format
        """
        # Get function metadata
        func_name = function.__name__
        func_description = inspect.getdoc(function) or ""
        
        # Get signature and type hints
        sig = inspect.signature(function)
        type_hints = get_type_hints(function)
        
        # Create parameters schema
        parameters = {
            "type": "OBJECT",
            "properties": {},
            "required": []
        }
        
        # Process parameters
        for param_name, param in sig.parameters.items():
            # Skip self for methods
            if param_name == "self":
                continue
                
            # Get parameter type
            param_type = type_hints.get(param_name, str)
            param_schema = GeminiUtils._python_type_to_schema(param_type)
            
            # Add to properties
            parameters["properties"][param_name] = param_schema
            
            # If no default value, it's required
            if param.default == inspect.Parameter.empty:
                parameters["required"].append(param_name)
        
        # Create function definition
        function_def = {
            "name": func_name,
            "description": func_description,
            "parameters": parameters
        }
        
        # Add return type if available
        if "return" in type_hints:
            return_type = type_hints["return"]
            if return_type is not None:
                function_def["response"] = GeminiUtils._python_type_to_schema(return_type)
        
        return function_def
    
    @staticmethod
    def _python_type_to_schema(python_type: Any) -> Dict[str, Any]:
        """
        Convert Python type to Gemini schema type.
        
        Args:
            python_type: Python type annotation
            
        Returns:
            Schema type in Gemini API format
        """
        # Handle primitive types
        if python_type == str:
            return {"type": "STRING"}
        elif python_type == int:
            return {"type": "INTEGER"}
        elif python_type == float:
            return {"type": "NUMBER"}
        elif python_type == bool:
            return {"type": "BOOLEAN"}
        
        # Handle Union/Optional types
        origin = get_origin(python_type)
        if origin is Union:
            args = get_args(python_type)
            # Handle Optional (Union with None)
            if type(None) in args:
                non_none_args = [arg for arg in args if arg is not type(None)]
                if len(non_none_args) == 1:
                    base_type = GeminiUtils._python_type_to_schema(non_none_args[0])
                    base_type["nullable"] = True
                    return base_type
            
            # General Union as anyOf
            schemas = [GeminiUtils._python_type_to_schema(arg) for arg in args 
                      if arg is not type(None)]
            return {"anyOf": schemas}
        
        # Handle List
        if origin is list or python_type == List:
            args = get_args(python_type)
            item_type = str  # Default item type
            if args:
                item_type = args[0]
            return {
                "type": "ARRAY",
                "items": GeminiUtils._python_type_to_schema(item_type)
            }
        
        # Handle Dict
        if origin is dict or python_type == Dict:
            return {"type": "OBJECT"}
        
        # Default to string for unknown types
        return {"type": "STRING"}
    
    @staticmethod
    def extract_function_calls(response: Any) -> List[Dict[str, Any]]:
        """
        Extract function calls from model response.
        
        Args:
            response: Model response
            
        Returns:
            List of function calls
        """
        # Parse response if it's a string
        if isinstance(response, str):
            try:
                response_data = json.loads(response)
            except json.JSONDecodeError:
                # Not JSON, no function calls
                return []
        elif isinstance(response, dict):
            response_data = response
        else:
            return []
        
        # Check for function calls in different formats
        function_calls = []
        
        # Format 1: functionCalls array
        if "functionCalls" in response_data:
            return response_data["functionCalls"]
            
        # Format 2: function_calls array
        if "function_calls" in response_data:
            return response_data["function_calls"]
            
        # Format 3: functionCall in candidates
        if "candidates" in response_data:
            for candidate in response_data.get("candidates", []):
                if isinstance(candidate, dict) and "functionCall" in candidate:
                    function_calls.append(candidate["functionCall"])
                    
        # Format 4: toolCalls in various schemas
        tool_calls = response_data.get("toolCalls", []) or response_data.get("tool_calls", [])
        if tool_calls:
            for tool in tool_calls:
                if isinstance(tool, dict) and "function" in tool:
                    function_info = tool["function"]
                    # Convert to standard format
                    function_call = {
                        "name": function_info.get("name"),
                        "args": function_info.get("arguments", {})
                    }
                    
                    # Parse args if it's a string
                    if isinstance(function_call["args"], str):
                        try:
                            function_call["args"] = json.loads(function_call["args"])
                        except json.JSONDecodeError:
                            function_call["args"] = {}
                            
                    function_calls.append(function_call)
        
        return function_calls